## Capítulo 9 - Classes Desbalanceadas

Se estiver classificando dados e as classes não estiverem relativamente balanceadas quanto ao tamanho, a distorção em direção às classes mais populares poderão transparecer em seu modelo. Por exemplo, se tiver 1 caso positivo e 99 casos negativos, poderá obter 99% de exatidão simplesmente classificando tudo como negativo. Há várias opções para lidar com 'classes desbalanceadas' (imbalanced classes).

### Use uma métrica diferente

Uma dica é usar uma medida que não seja a exatidão (accuracy) para calibrar os modelos (o AUC é uma boa opção). Precisão (precision) e recall também são ótimas opções quando os tamanhos dos alvos (targets) forem diferentes. No entanto, há outras opções a serem consideradas também.

### Algoritmos baseados em árvores e ensembles

Modelos baseados em árvore poderão ter um melhor desempenho conforme a distribuição da classe menor. Se os dados tiverem a tendência de estar agrupados, poderão ser mais facilmente classificados. 

Os métodos de ensemble podem ainda ajudar a extrair as classes minoritárias, Bagging e boosting são opções encontradas em modelos de árvore como florestas aleatórias (random forests) e o XGBoost (Gradient Boosting).

### Modelos de penalização

Mutos modelos de classificação do scikit-learn aceitam o parâmetro 'class_weight'. Defini-lo com 'balanced' tentará regularizar as classes minoritárias e incentivará o modelo a classificá-las corretamente.

Como alternativa, pode fazer uma busca em grade (grid search) e especificar as opções de peso, passando um dicionário que mapeie classes e pesos (dando pesos maiores às classes menores).

A biblioteca XGBoost (https://xgboost.readthedocs.io) tem um parâmetro 'max_delta_step', que prode ser definido com um valor entre 1 e 10 para deixar o passo de atualização mais conservador. Há também um parâmetro 'scale_pos_weight' que define a razão entre amostras negativas e positivas (para classes binárias). Além do mais, 'eval_metric' deve ser definido com 'auc', em vez de usar o valor default igual a 'error' para classificação.

O modelo KNN tem um parâmetro 'weights' que pode gerar distorção  em vizinhos mais próximos. Se as amostras da classe minoritária estiverem próximas, definir esse parâmetro com 'distance' poderá melhorar o desempenho.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)

In [2]:
# Caminho em Pasta
path = "datasets/titanic/titanic3.xls"
df = pd.read_excel(path)
orig_df = df

In [3]:
def tweak_titanic(df):
    df = df.drop(
        columns=[
            "name",
            "ticket",
            "home.dest",
            "boat",
            "body",
            "cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)
    return df
def get_train_test_X_y(
    df, y_col, size=0.3, std_cols=None
):
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, random_state=42
    )
    cols = X.columns
    num_cols = [
        "pclass",
        "age",
        "sibsp",
        "parch",
        "fare",
    ]
    fi = impute.IterativeImputer()

    fitted = fi.fit_transform(X_train[num_cols])
    X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(num_cols)})
    test_fit = fi.transform(X_test[num_cols])
    X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(num_cols)})
    if std_cols:
        std = preprocessing.StandardScaler()
        fitted = std.fit_transform(X_train[std_cols])
        X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(std_cols)})
        test_fit = std.transform(X_test[std_cols])
        X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(std_cols)})

    return X_train, X_test, y_train, y_test

ti_df = tweak_titanic(df)
std_cols = "pclass,age,sibsp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])



### Upsampling da minoria

Pode fazer um upsampling da classe minoritária de algumas maneiras. Segue uma implementação com o sklearn:

In [4]:
from sklearn.utils import resample

mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]
df_upsample = resample(
    surv_df, 
    replace=True,
    n_samples=len(death_df),
    random_state=42,
)
df = pd.concat([death_df, df_upsample])

In [5]:
df.survived.value_counts()

0    809
1    809
Name: survived, dtype: int64

A biblioteca 'imbalanced-learn' também pode ser usada para amostrar aleatoriamente com substituição:

In [6]:
!pip install imblearn

In [7]:
from imblearn.over_sampling import (
    RandomOverSampler,
)
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)
pd.Series(y_ros).value_counts()

0    809
1    809
Name: survived, dtype: int64

### Gerando dados de minorias

A biblioteca imbalanced-learn também pode gerar novas amostras das classes minoritárias com os algoritmos para amostragens como 'SMOTE (Synthetic Minority Oversampling Technique)' e 'ADASYN (Adaptive Synthetic)'. O SMOTE funciona selecionando um de seus k vizinhos mais próximos, conectando uma linha a um deles e selecionando um ponto nessa linha. O ADASYN é semelhante ao SMOT, mas gera mais amostras a partir daquelas cujo aprendizado é mais díficil. As classes em imbalanced-learn se chamam 'over_sampling.SMOTE' e 'over_sampling.ADASYN'

### Downsasmpling da minoria

Outro método para balancear classes é fazer o downsampling das classes majoritárias. Segue um exemplo com o sklearn

In [8]:
from sklearn.utils import resample

mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]
df_downsample = resample(
    death_df,
    replace=False,
    n_samples=len(surv_df),
    random_state=42,
)
df = pd.concat([surv_df, df_downsample])

In [9]:
df.survived.value_counts()

1    809
0    809
Name: survived, dtype: int64

###### DICA => Não use substituição quando fizer um downsampling

A biblioteca 'imbalanced-learn' também implementa diversos algoritmos de downsampling:

* ClusterCentroids => Essa classe utiliza k-means (k-médias) para sintetizar dados com os centróides.

* RandomUnderSampler => Essa classe seleciona amostras aleatoriamente.

* NearMiss => Essa classe faz um downsampling removendo amostras que estão próximas umas das outras.

* TomekLink => Essa classe reduz as amostras removendo aquelas que estão mais próximas entre si.

* EditedNearestNeighbours => Essa classe remove amostras que tenham vizinhos  que não estão na classe majoritária ou que estejam todos na mesma classe.

* RepeatedNearestNeighbours => Essa classe chama 'EditedNearestNeighbours' repetidamente.

* AllKNN => Essa classe é semelhante, mas aumenta o número de vizinhos mais próximos durante as iterações do downsampling.

* CondensedNearestNeighbour => Essa classe escolhe uma amostra da classe para downsampling e, em seguida, itera pelas outras amostras da classe; se KNN não fizer uma classificação incorreta, essa amostra será adicionada.

* OneSidedSelection => Essa classe remove amostras com ruído.

* NeighbourhoodCleaningRule => Essa classe usa os resultados de 'EditedNearestNeighbour', aplicando aí o KNN.

* InstanceHardnessThreshold => Essa classe faz o treinamento de um modelo e então remove as amostras com baixas probabilidades. 

Todas essas classes aceitam o método '.fit_resample'.

### Upsampling e depois downsampling

A biblioteca 'imbalanced-learn' implementa 'SMOTEEN' e 'SMOTETOMEK', que fazem um umpsampling e depois aplicam downsampling para limpar os dados.